In [2]:
import numpy as np
from collections import Counter
import json
from dahuffman import HuffmanCodec


In [3]:
# read json file that has all registers (outout of ViT of flextok) for all images stored
with open('all_registers_imagenet.json', 'r') as f:
    all_registers_imagenet = json.load(f)
    
# Suppose all_registers_imagenet is a list of lists or a 2D numpy array: shape [num_images, 256]
# If it's a list of lists, flatten it:
all_ids = np.array(all_registers_imagenet).flatten()
# (12800000 total num of registers) / (50000 num of images in dataset) = 256 registers per image
print(f"Total number of registers: {len(all_ids)}") 

Total number of registers: 12800000


In [ ]:
# Count frequencies
freq_counter = Counter(all_ids)

# To get the most common register IDs and their counts:
most_common = freq_counter.most_common(10)  # Top 10 most frequent
print("Top 10 most frequent register IDs and their counts:")
for reg_id, count in most_common:
    print(f"Register ID {reg_id}: {count} times")

[48883 37661 63028 58260 37859 34587 31578 34107 30828 62123]
Top 10 most frequent register IDs and their counts:
Register ID 1566: 5623 times
Register ID 1565: 4922 times
Register ID 1054: 4423 times
Register ID 1032: 4116 times
Register ID 520: 3880 times
Register ID 1024: 3392 times
Register ID 512: 3235 times
Register ID 62919: 3206 times
Register ID 60359: 2697 times
Register ID 1536: 2557 times


In [5]:
# To get the frequency (probability) of each register ID:
total = len(all_ids)
register_probs = {reg_id: count / total for reg_id, count in freq_counter.items()}

# Example: print the probability of the most frequent register
most_frequent_id = most_common[0][0]
print(f"Probability of most frequent register ({most_frequent_id}): {register_probs[most_frequent_id]:.4f}")

# Print the probabilities of the top 10 most frequent registers
print("Top 10 most frequent register IDs and their probabilities:")
for reg_id, count in most_common:
    prob = register_probs[reg_id]
    print(f"Register ID {reg_id}: Probability = {prob:.6f}")

Probability of most frequent register (1566): 0.0004
Top 10 most frequent register IDs and their probabilities:
Register ID 1566: Probability = 0.000439
Register ID 1565: Probability = 0.000385
Register ID 1054: Probability = 0.000346
Register ID 1032: Probability = 0.000322
Register ID 520: Probability = 0.000303
Register ID 1024: Probability = 0.000265
Register ID 512: Probability = 0.000253
Register ID 62919: Probability = 0.000250
Register ID 60359: Probability = 0.000211
Register ID 1536: Probability = 0.000200


In [53]:
# Get the list of probabilities
probs = np.array(list(register_probs.values()))

print("Number of unique registers", len(probs))  

# Variance of the probabilities
std = np.std(probs)
print(f"Standard deviation of register probabilities: {std:.6e}")

# shannon entropy is sum(-p * log2(p)) for all probabilities p for unique registers.
# here, we are comparing the entropy to max possible entropy, which is log2(1/num_unique_registers)
entropy = -np.sum(probs * np.log2(probs + 1e-12))
print(f"Entropy of register distribution: {entropy:.4f}")
uniform_distribution = -np.log2(1 / len(probs))
print(f"If the register IDs were uniformly distributed, the entropy would be:  {uniform_distribution:.4f}")

# Your data’s current entropy ≈ 15.615 bits/symbol =>
# (that's the ideal average. No coding system can do better on average than that, 
# per Shannon's source coding theorem.)
# If we don't apply any entropy coding, it is like assuming max entropy, where we 
# assign every register ID the same number of bits.
# so by applying Huffman coding, (or other entropy coding methods),
# the max reduction in bits/symbol is 0.35 bits/symbol

Number of unique registers 64000
Standard deviation of register probabilities: 1.251090e-05
Entropy of register distribution: 15.6154
If the register IDs were uniformly distributed, the entropy would be:  15.9658


In [ ]:
# here all_ids is a 1D numpy array of register IDs of all images from imagenet flattened
codec = HuffmanCodec.from_data(all_ids)

# 2. Print the symbol → Huffman bitcode table
codec.print_code_table()

# 3. Access the codebook directly
codebook = codec.get_code_table()  # returns dict: symbol → (bit_length, code_string)

In [ ]:
print(f"Number of unique registers in codebook: {len(codebook)}")
# get the lowest 2 register IDs directly

print(max(codebook.keys()), min(codebook.keys()))
# get the key which has the highest value in the codebook
min_key = min(codebook, key=lambda k: codebook[k][0])  #
print(f"Register ID with the longest code: {min_key} with length {codebook[min_key]} bits")

Number of unique registers in codebook: 64001
63999 _EOF
Register ID with the longest code: 1565 with length (11, 344) bits


In [44]:
# Baseline bits per symbol (fixed-length encoding)
# Ceiling is only for fixed-length encoding.
baseline_bits = np.ceil(-np.log2(1/len(register_probs)))

# Average bits per symbol with Huffman coding.
# Here, we don't use ceiling, because we are taking the average bits per symbol
# huffman already gives integer number of bits for each register ID.
average_bits_huffman = sum(
    register_probs[reg_id] * codebook[reg_id][0]  # Probability * Huffman bit length
    for reg_id in register_probs
)

# Reduction achieved
reduction = baseline_bits - average_bits_huffman

print(f"Baseline bits per symbol: {baseline_bits:.2f}")
print(f"Average bits per symbol with Huffman coding: {average_bits_huffman:.2f}")
print(f"Reduction achieved: {reduction:.2f} bits/symbol")

Baseline bits per symbol: 16.00
Average bits per symbol with Huffman coding: 15.64
Reduction achieved: 0.36 bits/symbol
